In [1]:
import numpy as np
from itertools import combinations, chain

In [69]:
def solve_exact_cover(U, subsets):
    """
    Solves the Exact Cover problem.
    
    Parameters:
    - U: A set of elements to be covered (universal set).
    - subsets: A list of subsets of U.
    
    Returns:
    - A list of subsets that form an exact cover of U, or an empty list if no solution exists.
    """
    def search(solution):
        # If no columns left, we found a solution
        if not columns:
            results.append(solution[:])
            return

        # Choose the column with the fewest rows
        c = min(columns, key=lambda col: len(matrix[col]))
        for row in list(matrix[c]):
            # Choose this row
            solution.append(row)
            covered = []
            for col in subsets[row]:
                for r in list(matrix[col]):
                    for c2 in subsets[r]:
                        if c2 != col:
                            matrix[c2].remove(r)
                    covered.append((col, r))
                del matrix[col]
                columns.remove(col)

            # Recur
            search(solution)

            # Backtrack
            for col, r in covered:
                if col not in matrix:
                    matrix[col] = set()
                    columns.add(col)
                matrix[col].add(r)
            solution.pop()

    # Create the matrix representation
    matrix = {}
    columns = set(U)
    for i, subset in enumerate(subsets):
        for element in subset:
            if element not in matrix:
                matrix[element] = set()
            matrix[element].add(i)

    results = []
    search([])
    return [subsets[i] for i in results[0]] if results else []


In [7]:
X = {1, 2, 3, 4}
Y = [{1, 2}, {2, 3}, {3, 4}, {4, 1}]
solution = solve_exact_cover(X, Y)
print(solution)

[{1, 2}, {3, 4}]


In [43]:
def covert_state_to_set(state):
    return set(np.nonzero(state.flatten())[0])

def get_minimal_state(state):
    axii = state.ndim
    for axis in range(axii):
        while 1 not in np.take(state, indices=[0], axis=axis):
            state = np.roll(state, -1, axis=axis)
    return state

def get_all_rotations(state):
    """
    Returns all unique 90-degree rotations of an n-dimensional numpy array.
    
    Args:
        array: n-dimensional numpy array
        
    Returns:
        List of rotated arrays
    """
    n = state.ndim
    rotations = []
    
    # Get all possible axis pairs for rotation
    axis_pairs = list(combinations(range(n), 2))
    
    def rotate_recursive(curr, depth=0):
        if depth == len(axis_pairs):
            if not any(np.array_equal(curr, r) for r in rotations):
                rotations.append(curr.copy())
            return
            
        # Try all 4 rotations for current axis pair
        temp = curr.copy()
        for _ in range(4):
            rotate_recursive(temp, depth + 1)
            temp = np.rot90(temp, axes=axis_pairs[depth])
    
    rotate_recursive(state)
    return rotations

def get_all_rolls(state, axis):
    states = []
    while 1 not in np.take(state, indices=[state.shape[axis] - 1], axis=axis):
        state = np.roll(state, 1, axis=axis)
        states.append(state)
    return states

def calc_all_states(state):
    axii = state.ndim
    states = []
    rotations = get_all_rotations(state)
    for rot in rotations:
        rot = get_minimal_state(rot)
        rolls = []
        rolls.append(rot)
        for axis in range(axii):
            axis_rolls = []
            for roll in rolls:
                axis_rolls += get_all_rolls(roll, axis)
            rolls += axis_rolls
        states += rolls
    return states


In [51]:
state = np.array([[1,1,1],[0,0,0],[0,0,0]])
states = calc_all_states(state)

print("States:")
for state_ in states:
    print(state_)
    print("")
    
print("State sets:")
state_sets = [covert_state_to_set(state) for state in states]
print(state_sets)

print("solutions:")
solutions = solve_exact_cover({0,1,2,3,4,5,6,7,8}, state_sets)
print(solutions)

States:
[[1 1 1]
 [0 0 0]
 [0 0 0]]

[[0 0 0]
 [1 1 1]
 [0 0 0]]

[[0 0 0]
 [0 0 0]
 [1 1 1]]

[[1 0 0]
 [1 0 0]
 [1 0 0]]

[[0 1 0]
 [0 1 0]
 [0 1 0]]

[[0 0 1]
 [0 0 1]
 [0 0 1]]

[[1 1 1]
 [0 0 0]
 [0 0 0]]

[[0 0 0]
 [1 1 1]
 [0 0 0]]

[[0 0 0]
 [0 0 0]
 [1 1 1]]

[[1 0 0]
 [1 0 0]
 [1 0 0]]

[[0 1 0]
 [0 1 0]
 [0 1 0]]

[[0 0 1]
 [0 0 1]
 [0 0 1]]

State sets:
[{0, 1, 2}, {3, 4, 5}, {8, 6, 7}, {0, 3, 6}, {1, 4, 7}, {8, 2, 5}, {0, 1, 2}, {3, 4, 5}, {8, 6, 7}, {0, 3, 6}, {1, 4, 7}, {8, 2, 5}]
solutions:
[[{0, 1, 2}, {3, 4, 5}, {8, 6, 7}], [{0, 3, 6}, {1, 4, 7}, {8, 2, 5}]]


In [ ]:
state = np.array([[[1,1,1,1,0],[0,1,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]])
states = calc_all_states(state)

print("States:")
for state_ in states:
    print(state_)
    print("")
    
print("State sets:")
state_sets = [covert_state_to_set(state) for state in states]
print(state_sets)

print("solutions:")
solutions = solve_exact_cover(set(range(125)), state_sets)
print(solutions)

States:
[[[1 1 1 1 0]
  [0 1 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 1 1 0]
  [0 1 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 1 1 0]
  [0 1 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [1 1 1 1 0]
  [0 1 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [1 1 1 1 0]
  [0 1 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [1 1 1 1 0]
  [0 1 0 0 0]]]

[[[0 1 1 1 1]
  [0 0 1 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 1 1 1 1]
  [

In [66]:
# [[{1, 2, 3, 4, 7}, {33, 6, 24, 25, 15}, {34, 36, 37, 40, 31}, {0, 18, 9, 10, 27}, {21, 22, 39, 12, 30}, {19, 20, 38, 11, 29}, {32, 5, 23, 13, 14}, {16, 17, 35, 8, 26}, {41, 42, 43, 44, 28}], [{1, 2, 3, 4, 7}, {17, 35, 8, 25, 26}, {34, 36, 37, 40, 31}, {0, 18, 9, 10, 27}, {16, 33, 6, 24, 15}, {21, 22, 39, 12, 30}, {19, 20, 38, 11, 29}, {32, 5, 23, 13, 14}, {41, 42, 43, 44, 28}], [{37, 40, 41, 42, 43}, {18, 36, 9, 27, 28}, {33, 6, 24, 25, 15}, {21, 39, 12, 30, 31}, {32, 5, 22, 23, 14}, {17, 34, 35, 26, 44}, {4, 7, 8, 10, 13}, {19, 20, 38, 11, 29}, {0, 1, 2, 3, 16}], [{37, 40, 41, 42, 43}, {33, 6, 24, 25, 15}, {21, 39, 12, 30, 31}, {20, 38, 11, 28, 29}, {32, 5, 22, 23, 14}, {17, 34, 35, 26, 44}, {4, 7, 8, 10, 13}, {18, 19, 36, 9, 27}, {0, 1, 2, 3, 16}]]
print (len(solutions[3]))
np.array([[[1,1,1,1,0],[0,1,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]).flatten()

9


array([1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])